<a href="https://colab.research.google.com/github/tubagokhan/GrinCH/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install py-rouge==1.1
!pip install sentence_transformers

In [9]:
import rouge
from sentence_transformers import SentenceTransformer
import numpy as np
from nltk import sent_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## ROUGE

In [10]:
def prepare_results(m, p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(m, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

def rougeEvaluation(all_hypothesis, all_references):

    for aggregator in ['Avg']:
        print('Evaluation with {}'.format(aggregator))
        apply_avg = aggregator == 'Avg'
        apply_best = aggregator == 'Best'

        evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                               max_n=4,
                               limit_length=False,
                               length_limit=1000,
                               length_limit_type='words',
                               apply_avg=apply_avg,
                               apply_best=apply_best,
                               alpha=0.2, # Default F1_score
                               weight_factor=1.2,
                               stemming=True)

        scores = evaluator.get_scores(all_hypothesis, all_references)

        for metric, results in sorted(scores.items(), key=lambda x: x[0]):
            if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
                for hypothesis_id, results_per_ref in enumerate(results):
                    nb_references = len(results_per_ref['p'])
                    for reference_id in range(nb_references):
                        print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                        print('\t' + prepare_results(metric,results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
                print()
            else:
                print(prepare_results(metric, results['p'], results['r'], results['f']))
        print()

## RESCORE

In [11]:

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def redundancyMeasure(corpus):
    sentences=sent_tokenize(corpus)
    length=len(sentences)

    #print("Sentence Count:"+ str(len(sentences)))

    if(len(sentences)<2):
      return 0

    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(sentences)

    sentenceGraph =np.zeros((length, length))
    for x in range(length):
        for y in range(length):
            if x>y:
                similarity= cosine(sentence_embeddings[x],sentence_embeddings[y])
                sentenceGraph[x][y]=abs(similarity)
    #print(sentenceGraph)
    SumElement=(np.concatenate(sentenceGraph).sum())
    divide=(length-1)*length/2

    similarityScore= SumElement/divide

    #similarityScore2 =np.mean(sentenceGraph, dtype = np.float64)
    #print(similarityScore2)

    return (round(similarityScore,2))

## READ FILES

In [12]:
# Save files

import json
import os

# Define the file path
file_path = "/content/GraphInClusterPubMedTextRanksavedDocumentsandSummaries.json"  # Replace with the desired file path

all_summary=[]
all_goldstandart=[]

# Check if the file already exists
if os.path.exists(file_path):
    # Read the existing data from the JSON file
    with open(file_path, "r") as file:
        existing_data = json.load(file)

    # Combine the existing data with the new data (lists)
    all_summary = existing_data.get("all_summary", [])
    all_goldstandart = existing_data.get("all_goldstandart", [])


In [13]:
rougeEvaluation(all_summary, all_goldstandart)

Evaluation with Avg
	rouge-1:	P: 36.67	R: 48.36	F1: 43.43
	rouge-2:	P: 12.57	R: 15.76	F1: 14.30
	rouge-3:	P:  6.48	R:  7.53	F1:  6.92
	rouge-4:	P:  4.23	R:  4.56	F1:  4.24
	rouge-l:	P: 34.33	R: 42.97	F1: 39.62
	rouge-w:	P: 16.42	R: 11.09	F1: 11.43



In [16]:
rougeEvaluation(all_summary[:100], all_goldstandart[:100])

Evaluation with Avg
	rouge-1:	P: 36.98	R: 47.70	F1: 42.84
	rouge-2:	P: 13.20	R: 15.50	F1: 14.13
	rouge-3:	P:  7.18	R:  7.58	F1:  7.01
	rouge-4:	P:  4.90	R:  4.75	F1:  4.43
	rouge-l:	P: 34.55	R: 42.53	F1: 39.22
	rouge-w:	P: 16.64	R: 10.90	F1: 11.22



In [14]:
NStart=0
documentCount=500
NFinish=NStart+documentCount

listOfSimilarityGoldStandart=[]
listOfSimilaritySummary=[]

for N in range (NStart,NFinish):
  goldSummary=all_goldstandart[N]
  goldSummarySimilarity=redundancyMeasure(goldSummary)
  listOfSimilarityGoldStandart.append(goldSummarySimilarity)

  summary=all_summary[N]
  summarySimilarity=redundancyMeasure(summary)
  listOfSimilaritySummary.append(summarySimilarity)
  print(N)

ReScoreGoldStandart=round(sum(listOfSimilarityGoldStandart)/len(listOfSimilarityGoldStandart),2)
ReScoreSummary=round(sum(listOfSimilaritySummary)/len(listOfSimilaritySummary),2)

print(ReScoreGoldStandart)
print(ReScoreSummary)

Sentence Count:7


Sentence Count:10
Sentence Count:11
Sentence Count:8
Sentence Count:5
Sentence Count:9
Sentence Count:10
Sentence Count:10
Sentence Count:7
Sentence Count:5
Sentence Count:5
Sentence Count:8
Sentence Count:11
Sentence Count:10
Sentence Count:9
Sentence Count:10
Sentence Count:10
Sentence Count:10
Sentence Count:2
Sentence Count:6
Sentence Count:6
Sentence Count:10
Sentence Count:15
Sentence Count:0
Sentence Count:10
Sentence Count:10
Sentence Count:8
Sentence Count:10
Sentence Count:8
Sentence Count:4
Sentence Count:9
Sentence Count:7
Sentence Count:6
Sentence Count:7
Sentence Count:7
Sentence Count:10
Sentence Count:7
Sentence Count:9
Sentence Count:6
Sentence Count:6
Sentence Count:4
Sentence Count:10
Sentence Count:8
Sentence Count:8
Sentence Count:8
Sentence Count:0
Sentence Count:12
Sentence Count:10
Sentence Count:3
Sentence Count:10
Sentence Count:11
Sentence Count:10
Sentence Count:7
Sentence Count:10
Sentence Count:3
Sentence Count:9
Sentence Count:10
Sentence Count:10
Sentenc

In [15]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')